In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# Step 1: Data Loading and Preprocessing
# Load the tweet data from a CSV file
df = pd.read_csv('tweets.csv')

# Filter out the relevant columns
df = df[['candidate', 'sentiment', 'text']]

# Preprocess the text data
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

df['text'] = df['text'].apply(preprocess_text)

# Encode the sentiment labels
label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

FileNotFoundError: [Errno 2] No such file or directory: 'tweets.csv'

In [3]:
# Step 2: Text Vectorization
# Tokenize and pad the sequences
tokenizer = Tokenizer(num_words=20000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['text'].values)
sequences = tokenizer.texts_to_sequences(df['text'].values)
padded_sequences = pad_sequences(sequences, maxlen=100)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, df['sentiment'], test_size=0.2, random_state=42)

NameError: name 'df' is not defined

In [4]:
# Step 3: Model Development
# Build the model
model = models.Sequential([
    layers.Embedding(input_dim=20000, output_dim=128, input_length=100),
    layers.SpatialDropout1D(0.2),
    layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Convert labels to categorical format for training
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=3)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


NameError: name 'y_train' is not defined

In [5]:
# Step 4: Model Training and Evaluation
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=64,
    validation_data=(X_val, y_val),
    verbose=2
)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=2)
print(f'Validation Accuracy: {val_accuracy}')


NameError: name 'X_train' is not defined

In [6]:
# Example usage: Predicting sentiment of a new tweet
def predict_sentiment(text):
    text = preprocess_text(text)
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=100)
    prediction = model.predict(padded_sequence)
    sentiment = np.argmax(prediction)
    return label_encoder.inverse_transform([sentiment])[0]

# Test prediction on a sample tweet
sample_tweet = "I love this candidate's policies!"
print(predict_sentiment(sample_tweet))

NameError: name 'preprocess_text' is not defined